In [1]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

# Load the data
store = pd.read_csv('store.csv')
train = pd.read_csv('train.csv', low_memory=False)

# Merge the DataFrames on the 'Store' column
trainStore = train.merge(store, on='Store').dropna()

# Encode categorical variables
label_encoder = LabelEncoder()
for column in trainStore.columns:
    if trainStore[column].dtype == 'object':
        trainStore[column] = label_encoder.fit_transform(trainStore[column])

# Split the dataframe into input features (X) and target variable (y)
X = trainStore.drop(['Sales', 'Date'], axis=1)  # Remove 'Date' column as it's not used
y = trainStore['Sales']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# MLPRegressor Model with GridSearch
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grid_mlp = {
    'hidden_layer_sizes': [(64,), (128,), (64, 32), (128, 64)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'batch_size': [32, 64],
    'max_iter': [300, 500, 1000],  # Increase max_iter
    'early_stopping': [True],  # Add early stopping
    'n_iter_no_change': [10],  # Number of iterations with no improvement to wait before stopping
    'verbose': [True]  # Print progress
}

with tqdm(desc='MLPRegressor', total=1) as pbar_mlp:
    grid_search_mlp = GridSearchCV(MLPRegressor(random_state=42), param_grid_mlp, cv=5, scoring='neg_mean_squared_error')
    grid_search_mlp.fit(X_train_scaled, y_train)
    best_mlp = grid_search_mlp.best_estimator_
    pbar_mlp.update(1)

# Predictions and Metrics
y_pred = best_mlp.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print('MLPRegressor Model Metrics:')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R2): {r2}')


Using device: cuda


MLPRegressor:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration 1, loss = 11071162.75637699
Validation score: 0.612673
Iteration 2, loss = 923560.22839196
Validation score: 0.903289
Iteration 3, loss = 580169.10303212
Validation score: 0.906661
Iteration 4, loss = 560287.94876791
Validation score: 0.909840
Iteration 5, loss = 542963.69098296
Validation score: 0.912482
Iteration 6, loss = 527569.01614583
Validation score: 0.914776
Iteration 7, loss = 514458.53552815
Validation score: 0.916738
Iteration 8, loss = 502970.56957107
Validation score: 0.918534
Iteration 9, loss = 493251.03744227
Validation score: 0.920041
Iteration 10, loss = 484951.68861655
Validation score: 0.921273
Iteration 11, loss = 477630.27666111
Validation score: 0.922413
Iteration 12, loss = 470779.71856041
Validation score: 0.923536
Iteration 13, loss = 464326.58242255
Validation score: 0.924569
Iteration 14, loss = 458334.77311526
Validation score: 0.925509
Iteration 15, loss = 452625.17062907
Validation score: 0.926373
Iteration 16, loss = 447408.88465663
Validation